# Association Rule Mining for 2019 Season

## Preprocess

In [13]:
import pandas as pd

batters = pd.read_csv("Batters_With_Clusters.csv")
pitchers = pd.read_csv("Pitchers_With_Clusters.csv")

In [14]:
print(batters.columns.tolist())
print(pitchers.columns.tolist())


['year', 'player_id', 'last_name, first_name', 'HitterType2_Group', 'HitterType2', 'ClusteringStats_Group', 'ClusteringStats']
['player_id', 'year', 'last_name, first_name', 'kmeans_quality_cluster', 'quality_archetype', 'impact_cluster', 'impact_archetype']


### Read play data

In [15]:
import pandas as pd

chunksize = 200_000
reader = pd.read_csv("./pitch_level_playbyplay_data/statcast_pitch_by_pitch_2019.csv", chunksize=chunksize)

dfs = []
for chunk in reader:
    dfs.append(chunk)          # or process chunk immediately
df = pd.concat(dfs, ignore_index=True)


In [16]:
print(df.columns.tolist())


['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value', 'launch_speed_a

In [17]:
print(df['events'].unique())
print(df['description'].unique()[:50])  # sample first 50
print(df['bb_type'].unique())


['strikeout' nan 'field_out' 'single' 'walk' 'force_out' 'home_run'
 'grounded_into_double_play' 'sac_bunt' 'double' 'hit_by_pitch' 'sac_fly'
 'intent_walk' 'field_error' 'fielders_choice_out' 'triple' 'double_play'
 'fielders_choice' 'catcher_interf' 'sac_fly_double_play'
 'strikeout_double_play' 'truncated_pa' 'sac_bunt_double_play'
 'triple_play']
['swinging_strike' 'foul' 'ball' 'called_strike' 'hit_into_play'
 'swinging_strike_blocked' 'blocked_ball' 'hit_by_pitch' 'foul_bunt'
 'foul_tip' 'automatic_ball' 'missed_bunt' 'pitchout' 'bunt_foul_tip']
[nan 'popup' 'fly_ball' 'ground_ball' 'line_drive']


In [18]:
import pandas as pd

def make_matchup(stand, p_throws):
    """
    stand: batter handedness ('L' or 'R')
    p_throws: pitcher handedness ('L' or 'R')
    """
    return f"{stand}vs{p_throws}"   # e.g. 'LvsR'

def bucket_count(balls, strikes):
    """
    Bucket balls/strikes into a small number of baseball-meaningful states.
    """
    # Full count first (since it's also two strikes)
    if balls == 3 and strikes == 2:
        return "full"

    # Two strikes (non-full counts)
    if strikes == 2:
        return "two_strikes"

    # Hitter clearly ahead
    if balls - strikes >= 2:
        return "hitter_ahead"

    # Pitcher clearly ahead
    if strikes > balls:
        return "pitcher_ahead"

    # Everything else (0-0, 1-1, 2-1, etc.)
    return "even"

def clean_outcome(events):
    """
    Use Statcast 'events' as the outcome label.
    If it's NaN or something weird, return NaN so we can drop it later.
    """
    if pd.isna(events):
        return pd.NA
    return str(events)


In [19]:
def build_plate_appearances(df_raw):
    """
    From pitch-by-pitch Statcast, keep only the final pitch of each PA.
    """
    # Ensure sort order within each PA
    df_sorted = df_raw.sort_values(['game_pk', 'at_bat_number', 'pitch_number'])

    # Last pitch in each (game_pk, at_bat_number) group = PA outcome pitch
    df_pa = (df_sorted
             .groupby(['game_pk', 'at_bat_number'], as_index=False)
             .tail(1)
             .copy())

    # Make sure 'year' column exists and is numeric
    if 'year' not in df_pa.columns and 'game_year' in df_pa.columns:
        df_pa['year'] = df_pa['game_year']
    df_pa['year'] = df_pa['year'].astype(int)

    return df_pa


In [20]:
def enrich_context(df_pa):
    """
    Add matchup (LvsR, etc.), count_bucket, and outcome from 'events'.
    """

    # Matchup
    df_pa['matchup'] = df_pa.apply(
        lambda row: make_matchup(row['stand'], row['p_throws']),
        axis=1
    )

    # Count bucket
    df_pa['count_bucket'] = df_pa.apply(
        lambda row: bucket_count(row['balls'], row['strikes']),
        axis=1
    )

    # Outcome
    df_pa['outcome'] = df_pa['events'].apply(clean_outcome)

    # Drop rows with missing critical info
    df_pa = df_pa.dropna(subset=['batter_arch', 'pitcher_arch', 'outcome'])

    return df_pa


In [21]:
def add_archetypes(
    df_pa,
    hitters_df,
    pitchers_df,
    batter_cluster_col="HitterType2",      # or "ClusteringStats"
    pitcher_cluster_col="impact_archetype" # or "quality_archetype"
):
    """
    Merge batter & pitcher archetypes into the PA-level DataFrame.
    """

    # --- Merge batters ---
    bat_cols = ['player_id', 'year', batter_cluster_col]
    hitters_small = hitters_df[bat_cols].rename(
        columns={'player_id': 'batter'}
    )

    df_pa = df_pa.merge(
        hitters_small,
        on=['batter', 'year'],
        how='left'
    )

    # --- Merge pitchers ---
    pit_cols = ['player_id', 'year', pitcher_cluster_col]
    pitchers_small = pitchers_df[pit_cols].rename(
        columns={'player_id': 'pitcher'}
    )

    df_pa = df_pa.merge(
        pitchers_small,
        on=['pitcher', 'year'],
        how='left',
        suffixes=('_bat', '_pit')
    )

    # Rename cluster columns to unified names
    df_pa = df_pa.rename(columns={
        batter_cluster_col: 'batter_arch',
        pitcher_cluster_col: 'pitcher_arch'
    })

    return df_pa


In [22]:
def build_transactions_df(df_pa):
    """
    Keep just the fields we want as "items" in each transaction.
    """
    df_tx = df_pa[['batter_arch', 'pitcher_arch', 'matchup', 'count_bucket', 'outcome']].copy()
    return df_tx


In [23]:
def encode_transactions(df_tx):
    """
    One-hot encode the transactional fields into a boolean 0/1 DataFrame.
    Columns will look like:
      - batter_arch_<value>
      - pitcher_arch_<value>
      - matchup_LvsR
      - count_bucket_two_strikes
      - outcome_strikeout
    """
    cols = ['batter_arch', 'pitcher_arch', 'matchup', 'count_bucket', 'outcome']

    encoded_parts = []
    for col in cols:
        dummies = pd.get_dummies(df_tx[col], prefix=col)
        encoded_parts.append(dummies)

    df_encoded = pd.concat(encoded_parts, axis=1)

    # Boolean is lighter than int
    df_encoded = df_encoded.astype(bool)

    return df_encoded


In [24]:
def preprocess_statcast(
    df_raw,
    hitters_df,
    pitchers_df,
    batter_cluster_col="HitterType2",
    pitcher_cluster_col="impact_archetype"
):
    # 1) Pitch → PA
    df_pa = build_plate_appearances(df_raw)

    # 2) Merge archetypes
    df_pa = add_archetypes(
        df_pa,
        hitters_df=hitters_df,
        pitchers_df=pitchers_df,
        batter_cluster_col=batter_cluster_col,
        pitcher_cluster_col=pitcher_cluster_col
    )

    # 3) Add matchup, count bucket, and outcome
    df_pa = enrich_context(df_pa)

    # 4) Build transaction-level
    df_tx = build_transactions_df(df_pa)

    # 5) One-hot encode
    df_encoded = encode_transactions(df_tx)

    return df_pa, df_tx, df_encoded


In [25]:
df_pa_2019, df_tx_2019, df_encoded_2019 = preprocess_statcast(
    df_raw=df,              # your big 2019 pitch-by-pitch DF
    hitters_df=batters,
    pitchers_df=pitchers,
    batter_cluster_col="HitterType2",        # or "ClusteringStats"
    pitcher_cluster_col="impact_archetype"   # or "quality_archetype"
)


In [29]:
df_pa_2019.to_csv("./2019/pa_2019.csv", index=False)
df_tx_2019.to_csv("./2019/tx_2019.csv", index=False)
df_encoded_2019.to_csv("./2019/encoded_2019.csv", index=False)

In [56]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

minsup_count = 40
minsup = minsup_count / len(df_encoded_2019)
minconf = 0.20

frequent_itemsets = fpgrowth(df_encoded_2019, min_support=minsup, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minconf)


In [65]:
minconf = 0.04
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minconf)


In [74]:
good_outcomes = [
    # 'outcome_single',
    # 'outcome_double',
    # 'outcome_triple',
    'outcome_home_run',
    # 'outcome_walk',
    # 'outcome_intent_walk',
    # 'outcome_hit_by_pitch',
    # 'outcome_field_error'
]


In [75]:

rules = rules[
    rules['consequents'].apply(
        lambda s: any(item in good_outcomes for item in s)
    )
]


In [76]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,ante_len
7107,(matchup_LvsR),(outcome_home_run),0.326365,0.041382,0.013312,0.040789,0.985689,1.0,-0.000193,0.999383,-0.021099,0.037559,-0.000618,0.181242,1
7111,(pitcher_arch_Sinker Liability Pitchers (Sinke...,(outcome_home_run),0.060551,0.041382,0.003059,0.050520,1.220842,1.0,0.000553,1.009625,0.192552,0.030939,0.009533,0.062222,1
7114,(count_bucket_even),(outcome_home_run),0.313329,0.041382,0.021459,0.068488,1.655044,1.0,0.008493,1.029100,0.576385,0.064394,0.028277,0.293531,1
7115,(batter_arch_Balanced Hitters),(outcome_home_run),0.506126,0.041382,0.020875,0.041245,0.996704,1.0,-0.000069,0.999858,-0.006651,0.039639,-0.000142,0.272851,1
7117,(matchup_RvsR),(outcome_home_run),0.397968,0.041382,0.016248,0.040828,0.986625,1.0,-0.000220,0.999423,-0.022022,0.038403,-0.000577,0.216737,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8122,"(count_bucket_pitcher_ahead, batter_arch_Balan...",(outcome_home_run),0.021306,0.041382,0.001045,0.049062,1.185599,1.0,0.000164,1.008077,0.159953,0.016958,0.008012,0.037161,3
8129,"(count_bucket_pitcher_ahead, pitcher_arch_Bala...",(outcome_home_run),0.008116,0.041382,0.000676,0.083333,2.013775,1.0,0.000340,1.045765,0.507540,0.013854,0.043763,0.049839,3
8133,"(count_bucket_pitcher_ahead, batter_arch_Power...",(pitcher_arch_Balanced Positive Arsenal (Mild ...,0.015772,0.019676,0.000676,0.042885,2.179529,1.0,0.000366,1.024249,0.549858,0.019452,0.023675,0.038630,2
8137,"(count_bucket_pitcher_ahead, pitcher_arch_Bala...",(outcome_home_run),0.010099,0.041382,0.000661,0.065449,1.581595,1.0,0.000243,1.025753,0.371479,0.013007,0.025106,0.040711,3


In [77]:
rules['ante_len'] = rules['antecedents'].apply(lambda x: len(x))


C:\Users\alexl\AppData\Local\Temp\ipykernel_16420\3112114842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rules['ante_len'] = rules['antecedents'].apply(lambda x: len(x))


In [85]:
max_len = rules['ante_len'].max()
print("Max antecedent size:", max_len)


Max antecedent size: 4


In [88]:
rules_sorted = rules.sort_values(
    by=[ 'lift', 'confidence'],
    ascending=[False, False]
)


In [89]:
rules_sorted = rules_sorted[rules_sorted['ante_len'] == 4]


In [91]:
rules_sorted.to_csv("./2019/rules_sorted_by_conf_support.csv", index=False)
